In [2]:
import time
from pathlib import Path

import vk
import os
from pyvis.network import Network
from dotenv import load_dotenv
import networkx as nx

In [3]:
load_dotenv(Path('config/.env'))

token = ''
try:
    token = os.environ['VK_ACCESS_TOKEN']
except KeyError:
    print("You need to enter your access token from https://vkhost.github.io/ into env VK_ACCESS_TOKEN")
    exit(1)

vk_api = vk.API(token, v='5.131')

In [4]:
def get_friends(id, depth):
    try:
        return [(friend_id, depth + 1) for friend_id in vk_api.friends.get(user_id=id)['items']]
    except Exception as e:
        print(e)
        return [] # если профиль закрыт или удален

In [ ]:
root_users = [243317609, 207227130]
max_depth = 3
max_friends = 100

nodes = set()
nodes_per_depth = [ set() for _ in range(max_depth) ]
edges = set()
used = set()

end_nodes = set()

queue = [(id, 0) for id in root_users]
while len(queue) != 0:
    node = queue.pop(0)
    if node[0] in used:
        continue
    
    friends = get_friends(*node)
    if (len(friends) > max_friends and node[1] > 0):
        continue
    nodes.add(node[0])
    nodes_per_depth[node[1]].add(node)

    for friend in friends:
        if (friend[1] < max_depth):
            queue.append(friend)
            
            edge = [node[0], friend[0]]
            edge.sort()
            edges.add(tuple(edge))
        else:
            end_nodes.add(node[0])

    used.add(node[0])
    time.sleep(0.2)

In [5]:
for end_node in end_nodes:
    friends = get_friends(node, max_depth - 1)
    for friend in friends:
        if friend in nodes:
            edge = [node[0], friend[0]]
            edge.sort()
            edges.add(tuple(edge))
        
        time.sleep(0.1)
    time.sleep(0.1)

In [6]:
networkx_graph = nx.Graph()
networkx_graph.add_nodes_from(nodes)
networkx_graph.add_edges_from(edges)

In [7]:
degree_centrality = list(nx.degree_centrality(networkx_graph).items())
closeness_centrality = list(nx.closeness_centrality(networkx_graph).items())
betweenness_centrality = list(nx.betweenness_centrality(networkx_graph).items())
eigenvector_centrality = list(nx.eigenvector_centrality(networkx_graph, max_iter=600).items())
pagerank = list(nx.pagerank(networkx_graph, alpha=0.85, max_iter=600).items())

def make_dict(nodes_with_metric):
    result = {}
    for index, element in nodes_with_metric:
        result[index] = element
    return result

degree_centrality_dict = make_dict(degree_centrality)
closeness_centrality_dict = make_dict(closeness_centrality)
betweenness_centrality_dict = make_dict(betweenness_centrality)
eigenvector_centrality_dict = make_dict(eigenvector_centrality)
pagerank_dict = make_dict(pagerank)

In [8]:
def make_pyvis_graph(metric_dict, min_metric, max_metric):
    pyvis_graph = Network(notebook=True, height=2000)

    for friend in nodes:
        min_size = 10
        size_factor = 30
        size = min_size + (size_factor * (metric_dict[friend] - min_metric) / (max_metric - min_metric))

        if (friend, 0) in nodes_per_depth[0]:
            color = 'green'
        if (friend, 1) in nodes_per_depth[1]:
            color = 'yellow'
        if (friend, 2) in nodes_per_depth[2]:
            color = 'red'

        pyvis_graph.add_node(friend, label=str(friend) + '\n' + str(metric_dict[friend]), color = color, size = size)

    for friend_pair in edges:
        if (friend_pair[0] in nodes and friend_pair[1] in nodes):
            pyvis_graph.add_edge(friend_pair[0], friend_pair[1], color = 'grey')    

    return pyvis_graph

In [ ]:
pyvis_degree_centrality_graph = make_pyvis_graph(degree_centrality_dict, min(degree_centrality_dict.values()), max(degree_centrality_dict.values()))
pyvis_closeness_centrality_graph = make_pyvis_graph(closeness_centrality_dict, min(closeness_centrality_dict.values()), max(closeness_centrality_dict.values()))
pyvis_betweenness_centrality_graph = make_pyvis_graph(betweenness_centrality_dict, min(betweenness_centrality_dict.values()), max(betweenness_centrality_dict.values()))
pyvis_eigenvector_centrality_graph = make_pyvis_graph(eigenvector_centrality_dict, min(eigenvector_centrality_dict.values()), max(eigenvector_centrality_dict.values()))
pyvis_pagerank_graph = make_pyvis_graph(pagerank_dict, min(pagerank_dict.values()), max(pagerank_dict.values()))

In [ ]:
pyvis_degree_centrality_graph.show_buttons(filter_=['physics'])
pyvis_closeness_centrality_graph.show_buttons(filter_=['physics'])
pyvis_betweenness_centrality_graph.show_buttons(filter_=['physics'])
pyvis_eigenvector_centrality_graph.show_buttons(filter_=['physics'])
pyvis_pagerank_graph.show_buttons(filter_=['physics'])

pyvis_degree_centrality_graph.show(str(Path('degree_centrality.html')))
pyvis_closeness_centrality_graph.show(str(Path('closeness_centrality.html')))
pyvis_betweenness_centrality_graph.show(str(Path('betweenness_centrality.html')))
pyvis_eigenvector_centrality_graph.show(str(Path('eigenvector_centrality.html')))
pyvis_pagerank_graph.show(str(Path('pagerank.html')))

In [ ]:
print(f'количетсво узлов: {len(nodes)}')
print(f'количетсво ребер: {len(edges)}')